# Analysis of the di-muon spectrum using data from the CMS detector

This analysis extracts the mass spectrum of di-muons produced in proton-proton collisions at sqrt(s)=7TeV data from the CMS experiment data recorded in 2012 during Run B and C. It is implemented using the [Julia](https://julialang.org/) language and can run in a Jupyter notebook. Python and C++ version of this analysis can be found [here](https://opendata.web.cern.ch/record/12342).

The spectrum is computed from the data by calculating the invariant mass of muon pairs with opposite charge. In the resulting plot, you are able to rediscover particle resonances in a wide energy range from the [eta meson](https://en.wikipedia.org/wiki/Eta_meson) at about 548 MeV up to the [Z boson](https://en.wikipedia.org/wiki/W_and_Z_bosons) at about 91 GeV.

The analysis code produces a plot with the dimuon spectrum. Note that the bump at 30 GeV is not a resonance but an effect of the data taking due to the used trigger. The technical description of the dataset can be found in the respective record linked below.

The result of this analysis can be compared with [an official result of the CMS collaboration using data taken in 2010](https://cds.cern.ch/record/1456510), see the plot below:

![](http://cds.cern.ch/record/1456510/files/pictures_samples_dimuonSpectrum_40pb-1_mod-combined.png)

# Dataset description

The dataset consists of the following columns.

| Column name | Data type | Description |
|-------------|-----------|-------------|
| `nMuon` | `unsigned int` | Number of muons in this event |
| `Muon_pt` | `float[nMuon]` | Transverse momentum of the muons (stored as an array of size `nMuon`) |
| `Muon_eta` | `float[nMuon]` | Pseudorapidity of the muons |
| `Muon_phi` | `float[nMuon]` | Azimuth of the muons |
| `Muon_mass` | `float[nMuon]` | Mass of the muons |
| `Muon_charge` | `int[nMuon]` | Charge of the muons (either 1 or -1) |

## Software prerequeries

This notebook needs a [Julia kernel](https://julialang.github.io/IJulia.jl/stable/) to be executed in Jupyter.

In addition it uses the Julia packages NBInclude (used to include an auxiliary notebook), UnROOT (used to read the input data file), Gafly (used for plotting). The following commands will install these two packages on the system if they are not installed yet.

In [1]:
import Pkg
Pkg.add("NBInclude")
Pkg.add("UnROOT")
Pkg.add("Gadfly")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
┌ Warning: The active manifest file at `/home/pgras/.julia/environments/v1.7/Manifest.toml` has an old format that is being maintained.
│ To update to the new format run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
└ @ Pkg.Types /home/pgras/git.d/julia/usr/share/julia/stdlib/v1.7/Pkg/src/manifest.jl:287
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /home/pgras/.julia/environments/v1.7/Manifest.toml:0
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.t

Import the packages used in this notebook:

In [2]:
using NBInclude
using UnROOT

## Some utilities we will need

Define four-momentum and histogram tools

In [3]:
@nbinclude("JuliaUtils.ipynb")

## The data file

To run this example, we need to download the CMS data file from [here](https://opendata.cern.ch/record/12341/files/Run2012BC_DoubleMuParked_Muons.root). If you don't have the command `wget` on your system you can download the file using your preferred web browser and save it with the name `Run2012BC_DoubleMuParked_Muons.root` in the directory containing this notebook.

The file to download is 1.2 GByte big.

In [4]:
run(`wget -c --progress dot:giga -O Run2012BC_DoubleMuParked_Muons.root "https://opendata.cern.ch/record/12341/files/Run2012BC_DoubleMuParked_Muons.root"`);

--2021-11-16 09:14:40--  https://opendata.cern.ch/record/12341/files/Run2012BC_DoubleMuParked_Muons.root
Resolving opendata.cern.ch (opendata.cern.ch)... 188.184.28.138, 188.184.93.89, 188.185.82.144, ...
Connecting to opendata.cern.ch (opendata.cern.ch)|188.184.28.138|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [5]:
fname = "Run2012BC_DoubleMuParked_Muons.root"

"Run2012BC_DoubleMuParked_Muons.root"

## The analysis

The function that will analyse the events. We select events with exactly two muons with opposite charges, compute the mass of the "dimuon" object formed by the two objects, and fill an histogram that will represent the dimuon mass spectrum.

In [6]:
function analyze_tree(t, maxevents = -1)
    bins = 30_000 # Number of bins in the histogram
    low = 0.25 # Lower edge of the histogram
    up = 300.0 # Upper edge of the histogram
    h = H1{Float64}(Axis(bins, low, up))
    for (ievt, evt) in enumerate(t)
        maxevents >= 0 && ievt > maxevents && break
        evt.nMuon ==2 || continue
        evt.Muon_charge[1] != evt.Muon_charge[2] || continue
        dimuon_mass = m(ptetaphim(evt.Muon_pt[1], evt.Muon_eta[1], evt.Muon_phi[1], evt.Muon_mass[1])
                        + ptetaphim(evt.Muon_pt[2], evt.Muon_eta[2], evt.Muon_phi[2], evt.Muon_mass[2]))
        hfill!(h, dimuon_mass)
    end
    h
end;

We will now run the function on the data stored in the input `fname` `ROOT` file. It will take about 40 seconds to run on the 61.5 million of events contained in the file.

In [7]:
t = LazyTree(ROOTFile(fname),"Events")
@time h = analyze_tree(t);

 32.669432 seconds (165.32 M allocations: 22.806 GiB, 12.62% gc time, 2.41% compilation time)


# Let's plot the result

In [1]:
import Gadfly as gf
p = gf.plot(x=xedges(h), y=vcat(h.sumw[2:end-1], [h.sumw[end-1]]), gf.Geom.step, 
        gf.Scale.x_log10(minvalue=0.25, maxvalue=300.), gf.Scale.y_log10,
        gf.Guide.xlabel("Dimuon mass [GeV/c²]"), gf.Guide.ylabel("Event count"))

LoadError: UndefVarError: xedges not defined

[Back to the presentation](../01-Julia-dream-JIII2021.ipynb)